<center><img src="https://unskript.com/assets/favicon.png" alt="unSkript.com" width="100" height="100">
<h1 id="unSkript-Runbooks">unSkript Runbooks</h1>
<div class="alert alert-block alert-success">
<h3 id="Objective">Objective</h3>
<br><strong style="color: #000000;"><em>Find and Delete Elastic Load Balancers that don't have any target groups or instances assosiated with them</em></strong></div>
</center>
<p>&nbsp;</p>
<center>
<h2 id="Release-Unattached-AWS-Elastic-IPs"><strong><u>Delete ELBs With No Target Groups or Instances</u></strong></h2>
</center>
<h1 id="Steps-Overview">Steps Overview</h1>
<p>1)<a href="#1" target="_self" rel="noopener"> Find AWS ELBs with no targets or instances</a><br>2)<a href="#2" target="_self" rel="noopener"> Delete Load Balancers</a></p>

In [ ]:
if region == None:
    region = ''
if elb_arns and not region:
    raise SystemExit("Provide a region for the ELB ARNs !")
if elb_names and not region:
    raise SystemExit("Provide a region for the ELB names !")
if elb_arns and elb_names and not region:
    raise SystemExit("Provide a region for the ELB ARNs and ELB names!")

<h3 id="Find-unattached-Elastic-IPs"><a id="1" target="_self" rel="nofollow"></a>Find AWS ELBs with no targets or instances</h3>
<p>Using unSkript's Find AWS ELBs with no targets or instances action, we will find ELBs that don't have any target groups in the case of Application Load Balancers's or&nbsp;Network Load Balancers's and Classic Load Balancers that have no&nbsp; instances associated to them.</p>
<blockquote>
<p>This action takes the following parameters: <code>region</code></p>
</blockquote>
<blockquote>
<p>This action captures the following output: <code>elbs_with_no_targets</code></p>
</blockquote>

In [ ]:
##
##  Copyright (c) 2023 unSkript, Inc
##  All rights reserved.
##
from pydantic import BaseModel, Field
from typing import Optional, Tuple
from unskript.legos.aws.aws_list_all_regions.aws_list_all_regions import aws_list_all_regions
from unskript.connectors.aws import aws_get_paginator
import pprint


from beartype import beartype
@beartype
def aws_find_elbs_with_no_targets_or_instances_printer(output):
    if output is None:
        return
    pprint.pprint(output)

@beartype
def aws_find_elbs_with_no_targets_or_instances(handle, region: str = "")->Tuple:
    """aws_find_elbs_with_no_targets_or_instances Returns details of Elb's with no target groups or instances

        :type handle: object
        :param handle: Object returned from task.validate(...).

        :type region: str
        :param region: AWS Region

        :rtype: Tuple of status, and details of ELB's with no targets or instances
    """
    result = []
    all_load_balancers = []
    all_regions = [region]
    if not region:
        all_regions = aws_list_all_regions(handle)
    for reg in all_regions:
        try:
            elbv2Client = handle.client('elbv2', region_name=reg)
            elbv2_response = aws_get_paginator(elbv2Client, "describe_load_balancers", "LoadBalancers")
            elbClient = handle.client('elb', region_name=reg)
            elb_response = elbClient.describe_load_balancers()
            for lb in elbv2_response:
                elb_dict = {}
                elb_dict["load_balancer_name"] = lb['LoadBalancerName']
                elb_dict["load_balancer_arn"] = lb['LoadBalancerArn']
                elb_dict["load_balancer_type"] = lb['Type']
                elb_dict["load_balancer_dns"] = lb['DNSName']
                elb_dict["region"] = reg
                all_load_balancers.append(elb_dict)
            for lb in elb_response['LoadBalancerDescriptions']:
                elb_dict = {}
                elb_dict["load_balancer_name"] = lb['LoadBalancerName']
                elb_dict["load_balancer_type"] = 'classic'
                elb_dict["load_balancer_dns"] = lb['DNSName']
                elb_dict["region"] = reg
                all_load_balancers.append(elb_dict)
        except Exception as e:
            pass
    for load_balancer in all_load_balancers:
        if load_balancer['load_balancer_type']=='network' or load_balancer['load_balancer_type']=='application':
            elbv2Client = handle.client('elbv2', region_name=load_balancer['region'])
            target_groups = elbv2Client.describe_target_groups(
                LoadBalancerArn=load_balancer['load_balancer_arn']
            )
            if len(target_groups['TargetGroups']) == 0:
                    elb_dict = {}
                    elb_dict["elb_arn"] = load_balancer['load_balancer_arn']
                    elb_dict["elb_name"] = load_balancer['load_balancer_name']
                    elb_dict["region"] = load_balancer['region']
                    elb_dict["type"] = load_balancer['load_balancer_type']
                    result.append(elb_dict)
        else:
            elbClient = handle.client('elb', region_name=load_balancer['region'])
            res = elbClient.describe_instance_health(
                LoadBalancerName=load_balancer['load_balancer_name'],
            )
            if len(res['InstanceStates'])==0:
                elb_dict = {}
                elb_dict["elb_name"] = load_balancer['load_balancer_name']
                elb_dict["region"] = load_balancer['region']
                elb_dict["type"] = load_balancer['load_balancer_type']
                result.append(elb_dict)
    if len(result) != 0:
        return (False, result)
    else:
        return (True, None)




task = Task(Workflow())
task.configure(inputParamsJson='''{
    "region": "region"
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "not elb_arns and not elb_names",
    "condition_result": true
    }''')

task.configure(outputName="elbs_with_no_targets")

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_find_elbs_with_no_targets_or_instances, lego_printer=aws_find_elbs_with_no_targets_or_instances_printer, hdl=hdl, args=args)

<h3 id="Create-List-of-unattached-Elastic-IPs">Create List of ELBs with no targets or instances</h3>
<p>This action filters regions that have no ELB's without targets and instances and creates a list of those that have them.</p>
<blockquote>
<p>This action takes the following parameters: <code>None</code></p>
</blockquote>
<blockquote>
<p>This action captures the following output: <code>elb_classic_list, elbv2_list</code></p>
</blockquote>

In [ ]:
elb_classic_list = []
elbv2_list = []
try:
    for res in elbs_with_no_targets:
        if type(res)==bool:
            if res == False:
                continue
        elif type(res)==list:
            if len(res)!=0:
                for elb in res:
                    if 'elb_arn' in elb.keys():
                        elbv2_list.append(elb)
                    else:
                        elb_classic_list.append(elb)
except Exception:
    if elb_arns:
        for arn in elb_arns:
            data_dict = {}
            data_dict["region"] = region[0]
            data_dict["elb_arn"] = arn
            elbv2_list.append(data_dict)
    if elb_names:
        for name in elb_names:
            data_dict = {}
            data_dict["region"] = region
            data_dict["elb_name"] = name
            elb_classic_list.append(data_dict)
print("Network/Application Load Balancers","\n",elbv2_list, "\n", "Classic Load Balancers", "\n", elb_classic_list)
task.configure(outputName="all_elbs_with_no_targets")

<h3 id="Delete-Load-Balancers"><a id="2" target="_self" rel="nofollow"></a>Delete Load Balancers</h3>
<p>This action deletes Network and Application ELBs found in Step 1.&nbsp;</p>
<blockquote>
<p>This action takes the following parameters:&nbsp;<code>region, elb_arn</code></p>
</blockquote>

In [ ]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##
from pydantic import BaseModel, Field
from typing import Optional, Dict
import pprint


from beartype import beartype
@beartype
def aws_delete_load_balancer_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_delete_load_balancer(handle, region: str, elb_arn: str) -> Dict:
    """aws_delete_load_balancer dict of loadbalancers info.

        :type region: string
        :param region: AWS Region.

        :type elb_arn: string
        :param elb_arn: load balancer ARNs.

        :rtype: dict of load balancers info.
    """
    try:
        ec2Client = handle.client('elbv2', region_name=region)
        response = ec2Client.delete_load_balancer(LoadBalancerArn=elb_arn)
        return response
    except Exception as e:
        raise Exception(e)


task = Task(Workflow())
task.configure(continueOnError=True)
task.configure(inputParamsJson='''{
    "elb_arn": "iter.get(\\"elb_arn\\")",
    "region": "iter.get(\\"region\\")"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "elbv2_list",
    "iter_parameter": ["elb_arn","region"]
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "len(elbv2_list)!=0",
    "condition_result": true
    }''')

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_delete_load_balancer, lego_printer=aws_delete_load_balancer_printer, hdl=hdl, args=args)

<h3 id="Delete-Load-Balancers"><a id="2" target="_self" rel="nofollow"></a>Delete Classic Load Balancers</h3>
<p>This action deletes Classic ELBs found in Step 1.&nbsp;</p>
<blockquote>
<p>This action takes the following parameters:&nbsp;<code>region, elb_name</code></p>
</blockquote>

In [ ]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##
from pydantic import BaseModel, Field
from typing import Optional, Dict
import pprint


def aws_delete_classic_load_balancer_printer(output):
    if output is None:
        return
    pprint.pprint(output)


def aws_delete_classic_load_balancer(handle, region: str, elb_name: str) -> Dict:
    """aws_delete_classic_load_balancer reponse of deleting a classic load balancer.

        :type region: string
        :param region: AWS Region.

        :type elb_arn: string
        :param elb_arn: Classic load balancer name.

        :rtype: dict of deleted load balancers reponse.
    """
    try:
        elblient = handle.client('elb', region_name=region)
        response = elblient.delete_load_balancer(LoadBalancerName=elb_name)
        return response
    except Exception as e:
        raise Exception(e)


task = Task(Workflow())
task.configure(inputParamsJson='''"iter.get(\\"elb_name\\")"
''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "elb_classic_list",
    "iter_parameter": ["elb_name","region"]
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "len(elb_classic_list)!=0",
    "condition_result": true
    }''')

task.configure(continueOnError=True)

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_delete_classic_load_balancer, lego_printer=aws_delete_classic_load_balancer_printer, hdl=hdl, args=args)

<h3 id="Conclusion">Conclusion</h3>
<p>In this Runbook, we were able to check for any AWS Elastic Load Balancers with no target groups or instances in our AWS account and release (remove) them in order to lower AWS costs. To view the full platform capabilities of unSkript please visit&nbsp;<a href="https://us.app.unskript.io" target="_blank" rel="noopener">us.app.unskript.io</a></p>